# Notebook: Transform dữ liệu từ tầng Bronze lên tầng Silver

## Mục tiêu
Transform dữ liệu orders từ tầng Bronze (raw JSON) lên tầng Silver (structured data) với các bước:
1. **Extract**: Parse JSON thành tabular format
2. **Transform**: Làm sạch và chuẩn hóa dữ liệu  
3. **Load**: Lưu vào Silver layer với schema tối ưu
4. **Document**: Tạo Data Dictionary

## Quy trình 8 bước chuẩn
1. **Import & Connect DB** - Thiết lập môi trường
2. **Load dữ liệu từ Bronze** - Đọc raw data
3. **JSON Structure Analysis** - Phân tích cấu trúc JSON
4. **Parse & Flatten JSON** - Chuyển đổi sang tabular
5. **Data Quality Check** - Đánh giá chất lượng dữ liệu
6. **Data Cleaning** - Làm sạch và chuẩn hóa
7. **Load vào Silver** - Lưu với schema tối ưu
8. **Data Dictionary** - Tạo tài liệu metadata

---


## Bước 1: Import & Connect DB

### Mục tiêu
- Import các thư viện cần thiết
- Load biến môi trường từ file `.env`
- Tạo kết nối tới MySQL database

### Kết nối Database
- `bronze_engine` → Đọc dữ liệu gốc từ schema **Bronze**
- `silver_engine` → Ghi dữ liệu đã làm sạch sang schema **Silver**

> **Lưu ý**: Đây là bước khởi tạo, chưa có quyết định gì, chỉ thiết lập môi trường làm việc.


In [34]:
import os
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER   = os.getenv("DB_USER")
DB_PASS   = os.getenv("DB_PASS")
DB_HOST   = os.getenv("DB_HOST")
DB_PORT   = os.getenv("DB_PORT")
DB_BRONZE = os.getenv("DB_BRONZE")
DB_SILVER = os.getenv("DB_SILVER")

# Tạo kết nối tới Bronze và Silver database
bronze_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_BRONZE}")
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")

print("✅ Đã kết nối Bronze và Silver database thành công")
print(f"📊 Bronze DB: {DB_BRONZE}")
print(f"📊 Silver DB: {DB_SILVER}")


✅ Đã kết nối Bronze và Silver database thành công
📊 Bronze DB: winner_bronze
📊 Silver DB: winner_silver


## Bước 2: Load dữ liệu từ Bronze

### Mục tiêu
- Lấy toàn bộ bảng `orders_raw` từ Bronze database
- Hiển thị thông tin cơ bản về dataset (shape, columns)
- Xem 1 record JSON mẫu để nắm cấu trúc dữ liệu

### Quy trình
1. **Load data**: Đọc từ `orders_raw` table
2. **Basic info**: Hiển thị shape và columns
3. **Sample JSON**: Parse và hiển thị 1 record mẫu

> **📌 Decision Point**: Từ JSON mẫu, ta sẽ quan sát để xác định có những nhóm trường nào (định danh, thông tin đơn hàng, khách hàng, sản phẩm, thanh toán...) và quyết định sẽ extract những field nào.

### Đặc điểm quan trọng của Orders
- **Fact Table**: Bảng sự kiện chính chứa các giao dịch
- **Complex JSON**: Nhiều nested objects (customer, items, payment, shipping...)
- **Multiple Entities**: Có thể extract nhiều bảng từ 1 JSON (orders, order_items, payments)
- **Business Critical**: Là trung tâm của hệ thống phân tích


In [35]:
# Load dữ liệu orders từ Bronze database
orders_df = pd.read_sql("SELECT * FROM orders_raw", bronze_engine)

# Hiển thị thông tin cơ bản về dataset
print("=== THÔNG TIN DATASET ===")
print(f"📊 Shape: {orders_df.shape}")
print(f"📋 Columns: {list(orders_df.columns)}")
print(f"💾 Memory usage: {orders_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị 1 record JSON mẫu để nắm cấu trúc
print("\n=== JSON STRUCTURE SAMPLE ===")
sample_json = json.loads(orders_df["raw_json"].iloc[10000])
print(json.dumps(sample_json, indent=2, ensure_ascii=False))


=== THÔNG TIN DATASET ===
📊 Shape: (40236, 4)
📋 Columns: ['shop_id', 'order_id', 'raw_json', 'extracted_at']
💾 Memory usage: 1276.99 MB

=== JSON STRUCTURE SAMPLE ===
{
  "time_send_partner": "2024-12-17T02:16:58",
  "creator": null,
  "note": "",
  "order_link": "https://pos.pages.fm/shop/230361475/order?order_id=90074161754541",
  "updated_at": "2024-12-24T02:39:14.521768",
  "warehouse_info": {
    "address": "Yên Xá",
    "affiliate_id": null,
    "commune_id": "1012335",
    "custom_id": null,
    "district_id": "10123",
    "ffm_id": null,
    "full_address": "Yên Xá, Xã Tân Triều, Huyện Thanh Trì, Hà Nội",
    "has_snappy_service": false,
    "name": "Kho Winner",
    "phone_number": "0344663171",
    "postcode": null,
    "province_id": "101",
    "settings": null
  },
  "buyer_total_amount": null,
  "page": {
    "id": "503588802831608",
    "name": "Veston Luxury",
    "username": null
  },
  "marketer": null,
  "partner": {
    "cod": 0,
    "count_of_delivery": 1,
    "cust

## Bước 3: JSON Structure Analysis

### Mục tiêu
- Scan toàn bộ keys trong JSON để hiểu đầy đủ cấu trúc
- Phân loại các trường theo mức độ quan trọng
- Chuẩn bị cho việc lựa chọn fields để extract

### Quy trình
1. **Scan keys**: Duyệt qua 200 records đầu để lấy tất cả keys
2. **Categorize**: Phân loại keys theo business value
3. **Decision**: Quyết định extract những field nào

> **Decision Point**: Từ danh sách keys, ta sẽ quyết định chọn những cột quan trọng để parse (ví dụ: order_id, customer_id, total_amount, order_date...) và bỏ các cột ít giá trị.

### Phân loại trường theo mức độ quan trọng:
- **High Priority**: Định danh, thông tin đơn hàng, khách hàng, tổng tiền
- **Medium Priority**: Thông tin giao hàng, thanh toán, trạng thái
- **Low Priority**: Logs, metadata, fields ít sử dụng

### Đặc biệt chú ý:
- **Nested Arrays**: items, payments có thể cần extract riêng
- **Customer Info**: Có thể join với dim_customers
- **Product Info**: Có thể join với dim_products


In [36]:
# Scan tất cả keys trong JSON để hiểu đầy đủ cấu trúc
print("=== SCANNING JSON KEYS ===")
all_keys = set()

# Scan 200 records đầu tiên để lấy tất cả keys có thể có
for raw in orders_df["raw_json"].head(500):
    d = json.loads(raw)
    all_keys.update(d.keys())

print(f"📊 Tổng số keys tìm thấy: {len(all_keys)}")
print(f"📋 Danh sách keys (sorted):")
for i, key in enumerate(sorted(all_keys), 1):
    print(f"{i:2d}. {key}")

# Phân tích cấu trúc nested
print(f"\n=== NESTED STRUCTURE ANALYSIS ===")
sample_data = json.loads(orders_df["raw_json"].iloc[0])

def analyze_nested_structure(data, prefix=""):
    """Phân tích cấu trúc nested objects và arrays"""
    structure = {}
    for key, value in data.items():
        if isinstance(value, dict):
            structure[f"{prefix}{key}"] = "object"
            structure.update(analyze_nested_structure(value, f"{prefix}{key}."))
        elif isinstance(value, list) and value:
            structure[f"{prefix}{key}"] = f"array[{len(value)}]"
            if value and isinstance(value[0], dict):
                structure.update(analyze_nested_structure(value[0], f"{prefix}{key}[0]."))
        else:
            structure[f"{prefix}{key}"] = type(value).__name__
    return structure

nested_structure = analyze_nested_structure(sample_data)
print(f"📊 Cấu trúc nested objects và arrays:")
for key, value_type in sorted(nested_structure.items()):
    if "." in key or "[" in key:  # Chỉ hiển thị nested structures
        print(f"  {key}: {value_type}")


=== SCANNING JSON KEYS ===
📊 Tổng số keys tìm thấy: 98
📋 Danh sách keys (sorted):
 1. account
 2. account_name
 3. activated_combo_products
 4. activated_promotion_advances
 5. ad_id
 6. ads_source
 7. advanced_platform_fee
 8. assigning_care
 9. assigning_care_id
10. assigning_seller
11. assigning_seller_id
12. bank_payments
13. bill_email
14. bill_full_name
15. bill_phone_number
16. botcake_info
17. buyer_total_amount
18. cash
19. charged_by_card
20. charged_by_momo
21. charged_by_qrpay
22. cod
23. conversation_id
24. creator
25. customer
26. customer_needs
27. customer_pay_fee
28. customer_referral_code
29. estimate_delivery_date
30. exchange_payment
31. exchange_value
32. fee_marketplace
33. histories
34. id
35. inserted_at
36. is_exchange_order
37. is_free_shipping
38. is_live_shopping
39. is_livestream
40. is_smc
41. items
42. items_length
43. last_editor
44. levera_point
45. link
46. link_confirm_order
47. marketer
48. marketplace_id
49. money_to_collect
50. note
51. note_image


## Bước 4: Parse & Flatten JSON

### Mục tiêu
- Tạo parsing function để extract 6 bảng từ JSON phức tạp
- Xử lý nested objects và arrays một cách cẩn thận
- Đảm bảo không mất dữ liệu và không tạo ra NULL values không mong muốn
- Test parsing function với sample data trước khi áp dụng toàn bộ dataset

### Thiết kế 6 bảng
1. **fact_orders** (25 fields) - Bảng chính với foreign keys
2. **dim_order_pages** (4 fields) - Thông tin page  
3. **dim_order_warehouses** (6 fields) - Thông tin kho
4. **dim_order_shipping** (6 fields) - Thông tin giao hàng
5. **dim_order_payments** (6 fields) - Thông tin thanh toán
6. **fact_order_items** (10 fields) - Chi tiết sản phẩm

### Quy trình
1. **Define parsing function**: Tạo function parse_order() cẩn thận
2. **Test với sample**: Test function với record có items
3. **Apply to dataset**: Áp dụng cho toàn bộ 40,236 records
4. **Validation**: Kiểm tra kết quả parsing

### Lưu ý quan trọng
- Xử lý NULL values cẩn thận - không để mất dữ liệu
- Đảm bảo surrogate keys được tạo đúng format
- Xử lý nested objects với error handling
- Test với record có items trước


In [37]:
def parse_order(row):
    """
    Parse JSON order data thành 6 bảng riêng biệt
    Extract tất cả thông tin cần thiết với xử lý nested objects
    """
    try:
        d = json.loads(row["raw_json"])
        
        # Lấy order_id để tạo surrogate keys
        order_id = d.get("id")
        if order_id is None:
            print(f"Warning: Missing order_id in record {row.name}")
            return None
        
        # 1. FACT_ORDERS - Bảng chính
        fact_orders = {
            # Định danh & Cơ bản
            "order_id": order_id,
            "system_id": d.get("system_id"),
            "shop_id": d.get("shop_id"),
            "order_link": d.get("order_link"),
            "link_confirm_order": d.get("link_confirm_order"),
            "order_currency": d.get("order_currency"),
            
            # Thông tin đơn hàng
            "total_price": d.get("total_price"),
            "total_price_after_sub_discount": d.get("total_price_after_sub_discount"),
            "total_discount": d.get("total_discount"),
            "total_quantity": d.get("total_quantity"),
            "items_length": d.get("items_length"),
            "tax": d.get("tax"),
            "shipping_fee": d.get("shipping_fee"),
            "money_to_collect": d.get("money_to_collect"),
            "is_free_shipping": d.get("is_free_shipping"),
            
            # Foreign Keys
            "customer_id": d.get("customer", {}).get("id") if d.get("customer") else None,
            "page_id": d.get("page", {}).get("id") if d.get("page") else None,
            "warehouse_id": d.get("warehouse_id"),
            "shipping_id": f"SHIP_{order_id}",  # Surrogate key
            "payment_id": f"PAY_{order_id}",    # Surrogate key
            
            # Trạng thái & Thời gian
            "status": d.get("status"),
            "status_name": d.get("status_name"),
            "inserted_at": d.get("inserted_at"),
            "updated_at": d.get("updated_at")
        }
        
        # 2. DIM_ORDER_PAGES - Thông tin page
        page_info = d.get("page", {})
        dim_order_pages = {
            "page_id": page_info.get("id"),
            "page_name": page_info.get("name"),
            "page_username": page_info.get("username"),
            "page_created_at": d.get("inserted_at")  # Sử dụng thời gian tạo order
        }
        
        # 3. DIM_ORDER_WAREHOUSES - Thông tin kho
        warehouse_info = d.get("warehouse_info", {})
        dim_order_warehouses = {
            "warehouse_id": d.get("warehouse_id"),
            "warehouse_name": warehouse_info.get("name"),
            "warehouse_address": warehouse_info.get("address"),
            "warehouse_full_address": warehouse_info.get("full_address"),
            "warehouse_phone_number": warehouse_info.get("phone_number"),
            "warehouse_commune_id": warehouse_info.get("commune_id"),
            "warehouse_province_id": warehouse_info.get("province_id")
        }
        
        # 4. DIM_ORDER_SHIPPING - Thông tin giao hàng
        shipping_address = d.get("shipping_address", {})
        dim_order_shipping = {
            "shipping_id": f"SHIP_{order_id}",  # Surrogate key
            "shipping_full_name": shipping_address.get("full_name"),
            "shipping_phone": shipping_address.get("phone_number"),
            "shipping_full_address": shipping_address.get("full_address"),
            "shipping_commune_id": shipping_address.get("commune_id"),
            "shipping_district_id": shipping_address.get("district_id"),
            "shipping_province_id": shipping_address.get("province_id")
        }
        
        # 5. DIM_ORDER_PAYMENTS - Thông tin thanh toán
        # Xác định payment method chính
        payment_method = "Unknown"
        if d.get("cod", 0) > 0:
            payment_method = "COD"
        elif d.get("cash", 0) > 0:
            payment_method = "Cash"
        elif d.get("charged_by_card", 0) > 0:
            payment_method = "Card"
        elif d.get("charged_by_momo", 0) > 0:
            payment_method = "MoMo"
        elif d.get("charged_by_qrpay", 0) > 0:
            payment_method = "QR"
        
        dim_order_payments = {
            "payment_id": f"PAY_{order_id}",  # Surrogate key
            "payment_method": payment_method,
            "payment_amount": d.get("total_price_after_sub_discount", 0),
            "cod": d.get("cod", 0),
            "cash": d.get("cash", 0),
            "charged_by_card": d.get("charged_by_card", 0),
            "charged_by_momo": d.get("charged_by_momo", 0),
            "charged_by_qrpay": d.get("charged_by_qrpay", 0)
        }
        
        # 6. FACT_ORDER_ITEMS - Chi tiết sản phẩm
        fact_order_items = []
        items = d.get("items", [])
        for i, item in enumerate(items):
            variation_info = item.get("variation_info", {})
            fact_order_items.append({
                "order_item_id": f"ITEM_{order_id}_{i}",  # Surrogate key
                "order_id": order_id,
                "product_id": item.get("product_id"),
                "variation_id": item.get("variation_id"),
                "item_quantity": item.get("quantity", 0),
                "item_price": variation_info.get("retail_price", 0),
                "item_discount": item.get("total_discount", 0),
                "return_quantity": item.get("return_quantity", 0),
                "returned_count": item.get("returned_count", 0),
                "product_name": variation_info.get("name"),
                "product_detail": variation_info.get("detail")
            })
        
        return {
            "fact_orders": fact_orders,
            "dim_order_pages": dim_order_pages,
            "dim_order_warehouses": dim_order_warehouses,
            "dim_order_shipping": dim_order_shipping,
            "dim_order_payments": dim_order_payments,
            "fact_order_items": fact_order_items
        }
        
    except Exception as e:
        print(f"Error parsing order {row.name}: {str(e)}")
        return None

print("✅ Parsing function defined successfully")


✅ Parsing function defined successfully


In [38]:
# Test parsing function với record có items
print("=== TESTING PARSING FUNCTION ===")

# Tìm record có items để test
test_record = None
for idx, row in orders_df.iterrows():
    sample_json = json.loads(row["raw_json"])
    items = sample_json.get("items", [])
    if len(items) > 0:
        test_record = row
        print(f"Found record {idx} with {len(items)} items")
        break

if test_record is not None:
    test_result = parse_order(test_record)
    
    if test_result:
        print(f"✅ Parsing function works successfully")
        print(f"📊 Fact orders fields: {len(test_result['fact_orders'])}")
        print(f"📊 Dim pages fields: {len(test_result['dim_order_pages'])}")
        print(f"📊 Dim warehouses fields: {len(test_result['dim_order_warehouses'])}")
        print(f"📊 Dim shipping fields: {len(test_result['dim_order_shipping'])}")
        print(f"📊 Dim payments fields: {len(test_result['dim_order_payments'])}")
        print(f"📊 Fact order items: {len(test_result['fact_order_items'])} items")
        
        # Hiển thị sample data
        print(f"\n=== SAMPLE FACT_ORDERS ===")
        for key, value in test_result['fact_orders'].items():
            print(f"  {key}: {value}")
        
        print(f"\n=== SAMPLE FACT_ORDER_ITEMS ===")
        for item in test_result['fact_order_items']:
            print(f"  Item: {item['product_name']} - Qty: {item['item_quantity']} - Price: {item['item_price']}")
    else:
        print("❌ Parsing function failed")
else:
    print("❌ No records with items found for testing")


=== TESTING PARSING FUNCTION ===
Found record 158 with 1 items
✅ Parsing function works successfully
📊 Fact orders fields: 24
📊 Dim pages fields: 4
📊 Dim warehouses fields: 7
📊 Dim shipping fields: 7
📊 Dim payments fields: 8
📊 Fact order items: 1 items

=== SAMPLE FACT_ORDERS ===
  order_id: 40458
  system_id: 40458
  shop_id: 230361475
  order_link: https://pos.pages.fm/shop/230361475/order?order_id=450370767843327
  link_confirm_order: https://order.pke.gg/tracking?id=RyvKXPdNWkGlh2Wk8g70JjQTyqX71E%2B1K5kpElWwPhZ0hZ5QgjMFwpCtgdQXxxFV4UuXGXw%2F2CD9AYXWej2DqwS6XyesHSg%3D
  order_currency: VND
  total_price: 629000
  total_price_after_sub_discount: 359000
  total_discount: 270000
  total_quantity: 1
  items_length: 1
  tax: 0
  shipping_fee: 0
  money_to_collect: 359000
  is_free_shipping: False
  customer_id: a9a32247-602c-42c9-af41-ce449cb1ebcc
  page_id: 519898014538188
  warehouse_id: 5f7aaa7d-ae3c-48b6-8f0e-04207afae074
  shipping_id: SHIP_40458
  payment_id: PAY_40458
  status: 6


In [39]:
# Apply parsing function to toàn bộ dataset
print("=== PARSING FULL DATASET ===")

# Khởi tạo lists để chứa dữ liệu từ tất cả orders
fact_orders_list = []
dim_order_pages_list = []
dim_order_warehouses_list = []
dim_order_shipping_list = []
dim_order_payments_list = []
fact_order_items_list = []

# Biến đếm để theo dõi tiến trình
total_records = len(orders_df)
error_count = 0
success_count = 0

print(f"Processing {total_records:,} records...")

# Duyệt qua từng record
for idx, row in orders_df.iterrows():
    if idx % 5000 == 0:
        print(f"Processing record {idx:,}/{total_records:,}... ({success_count:,} success, {error_count:,} errors)")
    
    try:
        # Parse order
        result = parse_order(row)
        
        if result is not None:
            # Thêm vào các list
            fact_orders_list.append(result["fact_orders"])
            dim_order_pages_list.append(result["dim_order_pages"])
            dim_order_warehouses_list.append(result["dim_order_warehouses"])
            dim_order_shipping_list.append(result["dim_order_shipping"])
            dim_order_payments_list.append(result["dim_order_payments"])
            fact_order_items_list.extend(result["fact_order_items"])  # extend vì có thể có nhiều items
            
            success_count += 1
        else:
            error_count += 1
            
    except Exception as e:
        print(f"Error processing record {idx}: {str(e)}")
        error_count += 1

print(f"\n=== PARSING COMPLETED ===")
print(f"✅ Successfully parsed: {success_count:,} records")
print(f"❌ Errors: {error_count:,} records")
print(f"📊 Total fact_orders: {len(fact_orders_list):,}")
print(f"📊 Total dim_order_pages: {len(dim_order_pages_list):,}")
print(f"📊 Total dim_order_warehouses: {len(dim_order_warehouses_list):,}")
print(f"📊 Total dim_order_shipping: {len(dim_order_shipping_list):,}")
print(f"📊 Total dim_order_payments: {len(dim_order_payments_list):,}")
print(f"📊 Total fact_order_items: {len(fact_order_items_list):,}")

# Tạo DataFrames từ lists
print(f"\n=== CREATING DATAFRAMES ===")
fact_orders_df = pd.DataFrame(fact_orders_list)
dim_order_pages_df = pd.DataFrame(dim_order_pages_list)
dim_order_warehouses_df = pd.DataFrame(dim_order_warehouses_list)
dim_order_shipping_df = pd.DataFrame(dim_order_shipping_list)
dim_order_payments_df = pd.DataFrame(dim_order_payments_list)
fact_order_items_df = pd.DataFrame(fact_order_items_list)

print(f"✅ Created 6 DataFrames successfully")
print(f"📊 fact_orders: {fact_orders_df.shape}")
print(f"📊 dim_order_pages: {dim_order_pages_df.shape}")
print(f"📊 dim_order_warehouses: {dim_order_warehouses_df.shape}")
print(f"📊 dim_order_shipping: {dim_order_shipping_df.shape}")
print(f"📊 dim_order_payments: {dim_order_payments_df.shape}")
print(f"📊 fact_order_items: {fact_order_items_df.shape}")

# Sample data để verify
print(f"\n=== SAMPLE DATA VERIFICATION ===")
print("Sample fact_orders:")
print(fact_orders_df.head(2))

print("\nSample fact_order_items:")
print(fact_order_items_df.head(3))


=== PARSING FULL DATASET ===
Processing 40,236 records...
Processing record 0/40,236... (0 success, 0 errors)
Processing record 5,000/40,236... (5,000 success, 0 errors)
Processing record 10,000/40,236... (10,000 success, 0 errors)
Processing record 15,000/40,236... (15,000 success, 0 errors)
Processing record 20,000/40,236... (20,000 success, 0 errors)
Processing record 25,000/40,236... (25,000 success, 0 errors)
Processing record 30,000/40,236... (30,000 success, 0 errors)
Processing record 35,000/40,236... (35,000 success, 0 errors)
Processing record 40,000/40,236... (40,000 success, 0 errors)

=== PARSING COMPLETED ===
✅ Successfully parsed: 40,236 records
❌ Errors: 0 records
📊 Total fact_orders: 40,236
📊 Total dim_order_pages: 40,236
📊 Total dim_order_warehouses: 40,236
📊 Total dim_order_shipping: 40,236
📊 Total dim_order_payments: 40,236
📊 Total fact_order_items: 46,611

=== CREATING DATAFRAMES ===
✅ Created 6 DataFrames successfully
📊 fact_orders: (40236, 24)
📊 dim_order_pages: 

## Bước 5: Data Quality Check

### Mục tiêu
- Phân tích chất lượng dữ liệu cho từng bảng trong 6 bảng đã parse
- Kiểm tra NULL values, duplicates, data types
- Phát hiện các vấn đề tiềm ẩn trước khi cleaning
- Đưa ra khuyến nghị cho bước cleaning tiếp theo

### Quy trình
1. **Null Analysis**: Phân tích tỷ lệ NULL theo từng cột
2. **Duplicate Check**: Kiểm tra duplicate records
3. **Data Type Analysis**: Phân tích kiểu dữ liệu
4. **Value Analysis**: Phân tích giá trị sample
5. **Relationship Check**: Kiểm tra mối quan hệ giữa các bảng
6. **Quality Summary**: Tổng kết chất lượng dữ liệu

### Lưu ý quan trọng
- Phân tích từng bảng riêng biệt
- Tập trung vào primary keys và foreign keys
- Đánh giá mức độ nghiêm trọng của các vấn đề
- Chuẩn bị cho bước cleaning


In [40]:
def analyze_dataframe_quality(df, table_name, primary_key):
    """
    Phân tích chất lượng dữ liệu cho một DataFrame
    """
    print(f"\n--- {table_name.upper()} ANALYSIS ---")
    print(f"Shape: {df.shape}")
    print(f"Columns: {len(df.columns)}")
    
    # 1. Null Analysis
    null_ratio = df.isnull().mean().sort_values(ascending=False)
    high_null_cols = null_ratio[null_ratio > 0.7].index.tolist()
    print(f"Columns with >70% null: {high_null_cols}")
    
    # 2. Duplicate Check
    if primary_key in df.columns:
        dup_count = df[primary_key].duplicated().sum()
        print(f"Duplicate {primary_key}: {dup_count}")
    
    # 3. Sample Values cho key columns
    key_columns = [col for col in df.columns if col in ['order_id', 'page_id', 'warehouse_id', 'product_id', 'total_price', 'status', 'payment_method']]
    for col in key_columns[:3]:
        if col in df.columns:
            print(f"{col} sample values: {df[col].dropna().unique()[:3].tolist()}")
    
    return high_null_cols

print("=== DATA QUALITY ANALYSIS FOR ALL TABLES ===")

# Analyze từng bảng
high_null_fact_orders = analyze_dataframe_quality(fact_orders_df, "fact_orders", "order_id")
high_null_pages = analyze_dataframe_quality(dim_order_pages_df, "dim_order_pages", "page_id")
high_null_warehouses = analyze_dataframe_quality(dim_order_warehouses_df, "dim_order_warehouses", "warehouse_id")
high_null_shipping = analyze_dataframe_quality(dim_order_shipping_df, "dim_order_shipping", "shipping_id")
high_null_payments = analyze_dataframe_quality(dim_order_payments_df, "dim_order_payments", "payment_id")
high_null_items = analyze_dataframe_quality(fact_order_items_df, "fact_order_items", "order_item_id")


=== DATA QUALITY ANALYSIS FOR ALL TABLES ===

--- FACT_ORDERS ANALYSIS ---
Shape: (40236, 24)
Columns: 24
Columns with >70% null: []
Duplicate order_id: 0
order_id sample values: [40616, 40615, 40614]
total_price sample values: [0, 629000, 649000]
page_id sample values: ['377626778776391', '492088430643389', '433231143199907']

--- DIM_ORDER_PAGES ANALYSIS ---
Shape: (40236, 4)
Columns: 4
Columns with >70% null: []
Duplicate page_id: 40162
page_id sample values: ['377626778776391', '492088430643389', '433231143199907']

--- DIM_ORDER_WAREHOUSES ANALYSIS ---
Shape: (40236, 7)
Columns: 7
Columns with >70% null: []
Duplicate warehouse_id: 40212
warehouse_id sample values: ['5f7aaa7d-ae3c-48b6-8f0e-04207afae074', 'a0b89882-3a6c-4167-8cad-37574a2be5f7', '2ff7f6ed-d985-49fd-9ba2-af8f78e7a5c5']

--- DIM_ORDER_SHIPPING ANALYSIS ---
Shape: (40236, 7)
Columns: 7
Columns with >70% null: []
Duplicate shipping_id: 0

--- DIM_ORDER_PAYMENTS ANALYSIS ---
Shape: (40236, 8)
Columns: 8
Columns with >70%

In [41]:
# Detailed Null Analysis
print("=== DETAILED NULL ANALYSIS ===")

def detailed_null_analysis(df, table_name):
    print(f"\n--- {table_name.upper()} NULL ANALYSIS ---")
    null_ratio = df.isnull().mean().sort_values(ascending=False)
    
    for col, ratio in null_ratio.items():
        if ratio > 0:
            print(f"  {col}: {ratio:.3f} ({ratio*100:.1f}%)")
        elif ratio == 0:
            print(f"  {col}: {ratio:.3f} ({ratio*100:.1f}%) - No nulls")

# Analyze null patterns cho từng bảng
detailed_null_analysis(fact_orders_df, "fact_orders")
detailed_null_analysis(dim_order_pages_df, "dim_order_pages")
detailed_null_analysis(dim_order_warehouses_df, "dim_order_warehouses")
detailed_null_analysis(dim_order_shipping_df, "dim_order_shipping")
detailed_null_analysis(dim_order_payments_df, "dim_order_payments")
detailed_null_analysis(fact_order_items_df, "fact_order_items")


=== DETAILED NULL ANALYSIS ===

--- FACT_ORDERS NULL ANALYSIS ---
  page_id: 0.000 (0.0%)
  order_id: 0.000 (0.0%) - No nulls
  shop_id: 0.000 (0.0%) - No nulls
  system_id: 0.000 (0.0%) - No nulls
  link_confirm_order: 0.000 (0.0%) - No nulls
  order_currency: 0.000 (0.0%) - No nulls
  total_price: 0.000 (0.0%) - No nulls
  order_link: 0.000 (0.0%) - No nulls
  total_price_after_sub_discount: 0.000 (0.0%) - No nulls
  total_discount: 0.000 (0.0%) - No nulls
  items_length: 0.000 (0.0%) - No nulls
  total_quantity: 0.000 (0.0%) - No nulls
  shipping_fee: 0.000 (0.0%) - No nulls
  money_to_collect: 0.000 (0.0%) - No nulls
  is_free_shipping: 0.000 (0.0%) - No nulls
  tax: 0.000 (0.0%) - No nulls
  customer_id: 0.000 (0.0%) - No nulls
  warehouse_id: 0.000 (0.0%) - No nulls
  shipping_id: 0.000 (0.0%) - No nulls
  payment_id: 0.000 (0.0%) - No nulls
  status: 0.000 (0.0%) - No nulls
  status_name: 0.000 (0.0%) - No nulls
  inserted_at: 0.000 (0.0%) - No nulls
  updated_at: 0.000 (0.0%) -

In [42]:
# Data Type and Value Analysis
print("=== DATA TYPE AND VALUE ANALYSIS ===")

def analyze_data_types(df, table_name):
    print(f"\n--- {table_name.upper()} DATA TYPES ---")
    for col in df.columns:
        dtype = df[col].dtype
        non_null_count = df[col].count()
        unique_count = df[col].nunique()
        
        print(f"  {col}: {dtype} | Non-null: {non_null_count:,} | Unique: {unique_count:,}")
        
        # Sample values cho string columns
        if dtype == 'object' and non_null_count > 0:
            sample_values = df[col].dropna().head(3).tolist()
            print(f"    Sample values: {sample_values}")

# Analyze data types cho từng bảng
analyze_data_types(fact_orders_df, "fact_orders")
analyze_data_types(dim_order_pages_df, "dim_order_pages")
analyze_data_types(dim_order_warehouses_df, "dim_order_warehouses")
analyze_data_types(dim_order_shipping_df, "dim_order_shipping")
analyze_data_types(dim_order_payments_df, "dim_order_payments")
analyze_data_types(fact_order_items_df, "fact_order_items")


=== DATA TYPE AND VALUE ANALYSIS ===

--- FACT_ORDERS DATA TYPES ---
  order_id: object | Non-null: 40,236 | Unique: 40,236
    Sample values: [40616, 40615, 40614]
  system_id: int64 | Non-null: 40,236 | Unique: 40,236
  shop_id: int64 | Non-null: 40,236 | Unique: 1
  order_link: object | Non-null: 40,236 | Unique: 40,236
    Sample values: ['https://pos.pages.fm/shop/230361475/order?order_id=360298824540530', 'https://pos.pages.fm/shop/230361475/order?order_id=270226832129888', 'https://pos.pages.fm/shop/230361475/order?order_id=180154838268782']
  link_confirm_order: object | Non-null: 40,236 | Unique: 40,236
    Sample values: ['https://order.pke.gg/tracking?id=RyvKXPdNWkGlh2Wk8g70JmtMD2KY08J6aPeuB%2FPPkM10hZ5QgjMFwpCtgdQXxxFV4UuXGXw%2F2IL9AYXWej2DqwS6XyesHSg%3D', 'https://order.pke.gg/tracking?id=RyvKXPdNWkGlh2Wk8g70JtXRy6zhKKJiMq5Lo%2B5cMAB0hZ5QgjMFwpCtgdQXxxFV4UuXGXw%2F2I39AYXWej2DqwS6XyesHSg%3D', 'https://order.pke.gg/tracking?id=RyvKXPdNWkGlh2Wk8g70JnY17RIlxpv5D5M%2BFnWKHNh0hZ

In [43]:
# Relationship Check
print("=== RELATIONSHIP CHECK ===")

def check_relationships():
    print("--- FOREIGN KEY RELATIONSHIPS ---")
    
    # Check fact_orders foreign keys
    print(f"fact_orders.order_id unique: {fact_orders_df['order_id'].nunique() == len(fact_orders_df)}")
    print(f"fact_orders.customer_id nulls: {fact_orders_df['customer_id'].isnull().sum()}")
    print(f"fact_orders.page_id nulls: {fact_orders_df['page_id'].isnull().sum()}")
    print(f"fact_orders.warehouse_id nulls: {fact_orders_df['warehouse_id'].isnull().sum()}")
    
    # Check fact_order_items foreign key to fact_orders
    orders_in_items = fact_order_items_df['order_id'].nunique()
    total_orders = fact_orders_df['order_id'].nunique()
    print(f"Orders in fact_order_items: {orders_in_items:,}")
    print(f"Total orders in fact_orders: {total_orders:,}")
    print(f"Orders with items: {orders_in_items/total_orders*100:.1f}%")
    
    # Check surrogate keys
    print(f"shipping_id unique: {dim_order_shipping_df['shipping_id'].nunique() == len(dim_order_shipping_df)}")
    print(f"payment_id unique: {dim_order_payments_df['payment_id'].nunique() == len(dim_order_payments_df)}")
    print(f"order_item_id unique: {fact_order_items_df['order_item_id'].nunique() == len(fact_order_items_df)}")
    
    # Check page_id consistency
    pages_in_orders = fact_orders_df['page_id'].nunique()
    pages_in_dim = dim_order_pages_df['page_id'].nunique()
    print(f"page_id in fact_orders: {pages_in_orders:,}")
    print(f"page_id in dim_order_pages: {pages_in_dim:,}")
    
    # Check warehouse_id consistency
    warehouses_in_orders = fact_orders_df['warehouse_id'].nunique()
    warehouses_in_dim = dim_order_warehouses_df['warehouse_id'].nunique()
    print(f"warehouse_id in fact_orders: {warehouses_in_orders:,}")
    print(f"warehouse_id in dim_order_warehouses: {warehouses_in_dim:,}")

check_relationships()


=== RELATIONSHIP CHECK ===
--- FOREIGN KEY RELATIONSHIPS ---
fact_orders.order_id unique: True
fact_orders.customer_id nulls: 0
fact_orders.page_id nulls: 10
fact_orders.warehouse_id nulls: 0
Orders in fact_order_items: 34,728
Total orders in fact_orders: 40,236
Orders with items: 86.3%
shipping_id unique: True
payment_id unique: True
order_item_id unique: True
page_id in fact_orders: 73
page_id in dim_order_pages: 73
warehouse_id in fact_orders: 24
warehouse_id in dim_order_warehouses: 24


In [44]:
# Quality Summary and Recommendations
print("=== QUALITY SUMMARY AND RECOMMENDATIONS ===")

def quality_summary():
    print("--- OVERALL QUALITY SUMMARY ---")
    
    # Collect all high-null columns
    all_high_null = []
    all_high_null.extend(high_null_fact_orders)
    all_high_null.extend(high_null_pages)
    all_high_null.extend(high_null_warehouses)
    all_high_null.extend(high_null_shipping)
    all_high_null.extend(high_null_payments)
    all_high_null.extend(high_null_items)
    
    print(f"Total tables analyzed: 6")
    print(f"Total columns with >70% null: {len(set(all_high_null))}")
    print(f"High-null columns: {set(all_high_null)}")
    
    print("\n--- RECOMMENDATIONS FOR CLEANING ---")
    
    # Recommendations based on analysis
    recommendations = []
    
    if high_null_fact_orders:
        recommendations.append(f"fact_orders: Consider dropping columns {high_null_fact_orders}")
    
    if high_null_pages:
        recommendations.append(f"dim_order_pages: Consider dropping columns {high_null_pages}")
    
    if high_null_warehouses:
        recommendations.append(f"dim_order_warehouses: Consider dropping columns {high_null_warehouses}")
    
    if high_null_shipping:
        recommendations.append(f"dim_order_shipping: Consider dropping columns {high_null_shipping}")
    
    if high_null_payments:
        recommendations.append(f"dim_order_payments: Consider dropping columns {high_null_payments}")
    
    if high_null_items:
        recommendations.append(f"fact_order_items: Consider dropping columns {high_null_items}")
    
    # General recommendations
    recommendations.extend([
        "Convert datetime columns to proper datetime type",
        "Convert numeric columns to proper numeric types",
        "Handle NULL values in foreign keys appropriately",
        "Standardize string columns (strip, title case)",
        "Remove duplicates if any found"
    ])
    
    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")
    
    print(f"\n--- NEXT STEPS ---")
    print("1. Apply data cleaning based on recommendations")
    print("2. Define proper schema for database loading")
    print("3. Load cleaned data to Silver database")
    print("4. Generate data dictionary")

quality_summary()


=== QUALITY SUMMARY AND RECOMMENDATIONS ===
--- OVERALL QUALITY SUMMARY ---
Total tables analyzed: 6
Total columns with >70% null: 0
High-null columns: set()

--- RECOMMENDATIONS FOR CLEANING ---
1. Convert datetime columns to proper datetime type
2. Convert numeric columns to proper numeric types
3. Handle NULL values in foreign keys appropriately
4. Standardize string columns (strip, title case)
5. Remove duplicates if any found

--- NEXT STEPS ---
1. Apply data cleaning based on recommendations
2. Define proper schema for database loading
3. Load cleaned data to Silver database
4. Generate data dictionary


## Bước 6: Data Cleaning & Transformation

### Mục tiêu
- Áp dụng các khuyến nghị từ Data Quality Analysis
- Clean và transform dữ liệu cho 6 bảng
- Chuẩn bị dữ liệu sạch để load vào Silver database

### Quy trình Cleaning
1. **Drop high-null columns** (nếu có)
2. **Datetime conversion** - Convert các cột datetime
3. **Numeric conversion** - Convert các cột numeric
4. **String standardization** - Clean và standardize strings
5. **NULL handling** - Xử lý NULL values phù hợp
6. **Duplicate removal** - Loại bỏ duplicates

### Lưu ý quan trọng
- Giữ nguyên data types cho status, shipping_id, payment_id
- Xử lý NULL values cẩn thận cho foreign keys
- Đảm bảo không mất dữ liệu quan trọng
- Test cleaning function trước khi áp dụng toàn bộ


In [45]:
def drop_high_null_cols(df, threshold=0.7):
    """
    Drop columns with null ratio > threshold
    """
    null_ratio = df.isnull().mean()
    cols_to_drop = null_ratio[null_ratio > threshold].index.tolist()
    
    if cols_to_drop:
        print(f"Dropping columns with >{threshold*100}% null: {cols_to_drop}")
        return df.drop(columns=cols_to_drop, errors="ignore")
    else:
        print(f"No columns with >{threshold*100}% null to drop")
        return df

def clean_dataframe(df, table_name, primary_key=None):
    """
    Clean và transform một DataFrame
    """
    print(f"\n--- CLEANING {table_name.upper()} ---")
    original_shape = df.shape
    
    # 1. Drop high-null columns
    df_clean = drop_high_null_cols(df, threshold=0.7)
    
    # 2. Datetime conversion - CHỈ cho các cột thực sự là datetime
    datetime_cols = [col for col in df_clean.columns if col in ['inserted_at', 'updated_at', 'page_created_at']]
    for col in datetime_cols:
        if col in df_clean.columns:
            print(f"Converting {col} to datetime...")
            df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")
    
    # 3. Numeric conversion - TRÁNH các cột status, shipping_id, payment_id
    numeric_cols = [col for col in df_clean.columns if any(x in col.lower() for x in ['price', 'quantity', 'discount', 'fee', 'amount', 'count']) and col not in ['status', 'status_name', 'shipping_id', 'payment_id']]
    for col in numeric_cols:
        if col in df_clean.columns:
            print(f"Converting {col} to numeric...")
            df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")
    
    # 4. String normalization - BAO GỒM status_name, shipping_id, payment_id
    string_cols = [col for col in df_clean.columns if any(x in col.lower() for x in ['name', 'username', 'address', 'method', 'detail', 'link', 'currency']) or col in ['status_name', 'shipping_id', 'payment_id']]
    for col in string_cols:
        if col in df_clean.columns:
            print(f"Standardizing {col}...")
            # Xử lý None value trước khi convert to string
            df_clean[col] = df_clean[col].fillna('').astype(str).str.strip()
            # Replace empty string với None nếu cần
            df_clean[col] = df_clean[col].replace('', None)
    
    # 5. Xử lý đặc biệt cho status - giữ nguyên integer
    if 'status' in df_clean.columns:
        print("Keeping status as integer...")
        df_clean['status'] = pd.to_numeric(df_clean['status'], errors="coerce").fillna(0).astype(int)
    
    # 6. Remove duplicates (chỉ cho dim tables)
    if primary_key and primary_key in df_clean.columns:
        dup_count = df_clean[primary_key].duplicated().sum()
        if dup_count > 0:
            print(f"Removing {dup_count} duplicates...")
            df_clean = df_clean.drop_duplicates(subset=[primary_key], keep='last')
    
    print(f"Shape: {original_shape} → {df_clean.shape}")
    return df_clean

print("✅ Cleaning functions defined successfully")


✅ Cleaning functions defined successfully


In [46]:
# Apply cleaning to all 6 DataFrames
print("=== APPLYING DATA CLEANING ===")

# Clean từng bảng
fact_orders_clean = clean_dataframe(fact_orders_df, "fact_orders", "order_id")
dim_order_pages_clean = clean_dataframe(dim_order_pages_df, "dim_order_pages", "page_id")
dim_order_warehouses_clean = clean_dataframe(dim_order_warehouses_df, "dim_order_warehouses", "warehouse_id")
dim_order_shipping_clean = clean_dataframe(dim_order_shipping_df, "dim_order_shipping", "shipping_id")
dim_order_payments_clean = clean_dataframe(dim_order_payments_df, "dim_order_payments", "payment_id")
fact_order_items_clean = clean_dataframe(fact_order_items_df, "fact_order_items", "order_item_id")

print(f"\n=== CLEANING COMPLETED ===")
print(f"✅ All 6 tables cleaned successfully")

# Summary of cleaned data
tables_clean = {
    "fact_orders": fact_orders_clean,
    "dim_order_pages": dim_order_pages_clean,
    "dim_order_warehouses": dim_order_warehouses_clean,
    "dim_order_shipping": dim_order_shipping_clean,
    "dim_order_payments": dim_order_payments_clean,
    "fact_order_items": fact_order_items_clean
}

print(f"\n=== CLEANED DATA SUMMARY ===")
for table_name, df in tables_clean.items():
    print(f"{table_name}: {df.shape[0]:,} rows, {df.shape[1]} columns")


=== APPLYING DATA CLEANING ===

--- CLEANING FACT_ORDERS ---
No columns with >70.0% null to drop
Converting inserted_at to datetime...
Converting updated_at to datetime...
Converting total_price to numeric...
Converting total_price_after_sub_discount to numeric...
Converting total_discount to numeric...
Converting total_quantity to numeric...
Converting shipping_fee to numeric...
Standardizing order_link...
Standardizing link_confirm_order...
Standardizing order_currency...
Standardizing shipping_id...
Standardizing payment_id...
Standardizing status_name...
Keeping status as integer...
Shape: (40236, 24) → (40236, 24)

--- CLEANING DIM_ORDER_PAGES ---
No columns with >70.0% null to drop
Converting page_created_at to datetime...
Standardizing page_name...
Standardizing page_username...
Removing 40162 duplicates...
Shape: (40236, 4) → (74, 4)

--- CLEANING DIM_ORDER_WAREHOUSES ---
No columns with >70.0% null to drop
Standardizing warehouse_name...
Standardizing warehouse_address...
Stan

In [47]:
# Verification of cleaned data
print("=== VERIFICATION OF CLEANED DATA ===")

def verify_cleaned_data():
    print("--- DATA TYPE VERIFICATION ---")
    
    # Check fact_orders
    print(f"\nfact_orders data types:")
    for col in ['order_id', 'status', 'total_price', 'inserted_at']:
        if col in fact_orders_clean.columns:
            print(f"  {col}: {fact_orders_clean[col].dtype}")
    
    # Check dim_order_pages
    print(f"\ndim_order_pages data types:")
    for col in ['page_id', 'page_name', 'page_username']:
        if col in dim_order_pages_clean.columns:
            print(f"  {col}: {dim_order_pages_clean[col].dtype}")
    
    # Check fact_order_items
    print(f"\nfact_order_items data types:")
    for col in ['order_item_id', 'item_quantity', 'item_price', 'product_name']:
        if col in fact_order_items_clean.columns:
            print(f"  {col}: {fact_order_items_clean[col].dtype}")
    
    print("\n--- SAMPLE DATA VERIFICATION ---")
    print("Sample fact_orders:")
    print(fact_orders_clean[['order_id', 'status', 'total_price', 'customer_id']].head(2))
    
    print("\nSample fact_order_items:")
    print(fact_order_items_clean[['order_item_id', 'order_id', 'item_quantity', 'item_price']].head(3))

verify_cleaned_data()


=== VERIFICATION OF CLEANED DATA ===
--- DATA TYPE VERIFICATION ---

fact_orders data types:
  order_id: object
  status: int64
  total_price: int64
  inserted_at: datetime64[ns]

dim_order_pages data types:
  page_id: object
  page_name: object
  page_username: object

fact_order_items data types:
  order_item_id: object
  item_quantity: int64
  item_price: int64
  product_name: object

--- SAMPLE DATA VERIFICATION ---
Sample fact_orders:
  order_id  status  total_price                           customer_id
0    40616       0            0  e906c5c7-ea19-42e5-a971-57cf09c94417
1    40615       0            0  6c5b40d2-ae42-4e13-8019-f3f9127d75a0

Sample fact_order_items:
  order_item_id order_id  item_quantity  item_price
0  ITEM_40458_0    40458              1      629000
1  ITEM_40455_0    40455              1      649000
2  ITEM_40454_0    40454              1      649000


## Bước 7: Load vào Silver Database

### Mục tiêu
- Load 6 bảng đã clean vào Silver database
- Định nghĩa schema mapping cho từng bảng
- Đảm bảo data types phù hợp với MySQL
- Load theo thứ tự: dim tables trước, fact tables sau

### Quy trình
1. **Schema Definition** - Định nghĩa SQLAlchemy types
2. **Load Dim Tables** - Load các dimension tables trước
3. **Load Fact Tables** - Load các fact tables sau
4. **Verification** - Kiểm tra kết quả load

### Lưu ý quan trọng
- Sử dụng explicit SQLAlchemy types
- Load dim tables trước để đảm bảo foreign key references
- Handle large datasets với chunking nếu cần
- Verify count và sample data sau khi load


In [48]:
from sqlalchemy.types import BigInteger, Integer, Float, DateTime, Text, VARCHAR, Boolean

def create_schema_mapping(df, table_name):
    """
    Tạo schema mapping cho một DataFrame
    """
    dtype_mapping = {}
    
    for col in df.columns:
        # ID fields - VARCHAR(100)
        if col.endswith('_id') and col not in ['shop_id', 'system_id']:
            dtype_mapping[col] = VARCHAR(100)
        
        # Name fields - VARCHAR(255)
        elif 'name' in col.lower() or 'username' in col.lower():
            dtype_mapping[col] = VARCHAR(255)
        
        # Address fields - VARCHAR(500)
        elif 'address' in col.lower():
            dtype_mapping[col] = VARCHAR(500)
        
        # Text fields - Text()
        elif 'detail' in col.lower() or 'method' in col.lower():
            dtype_mapping[col] = Text()
        
        # Numeric fields
        elif col in ['shop_id', 'system_id']:
            dtype_mapping[col] = BigInteger()
        
        # Integer fields
        elif any(x in col.lower() for x in ['quantity', 'count', 'status']):
            dtype_mapping[col] = Integer()
        
        # Float fields
        elif any(x in col.lower() for x in ['price', 'discount', 'fee', 'amount']):
            dtype_mapping[col] = Float()
        
        # DateTime fields
        elif 'at' in col.lower() or 'date' in col.lower():
            dtype_mapping[col] = DateTime()
        
        # Boolean fields
        elif col.startswith('is_'):
            dtype_mapping[col] = Boolean()
        
        # Default - VARCHAR(255)
        else:
            dtype_mapping[col] = VARCHAR(255)
    
    print(f"Schema mapping for {table_name}: {len(dtype_mapping)} columns")
    return dtype_mapping

def load_to_silver(df, table_name, dtype_mapping):
    """
    Load DataFrame vào Silver database
    """
    try:
        df.to_sql(
            table_name,
            con=silver_engine,
            if_exists="replace",  # Ghi đè dữ liệu cũ
            index=False,
            dtype=dtype_mapping
        )
        
        print(f"✅ Đã load {df.shape[0]:,} records vào Silver: {table_name}")
        
        # Verify load
        verification_query = f"SELECT COUNT(*) as count FROM {table_name}"
        verification_result = pd.read_sql(verification_query, silver_engine)
        loaded_count = verification_result['count'].iloc[0]
        
        print(f"✅ Verification: {loaded_count:,} records trong database")
        return True
        
    except Exception as e:
        print(f"❌ Error loading {table_name} to Silver: {str(e)}")
        return False

print("✅ Schema mapping and load functions defined successfully")


✅ Schema mapping and load functions defined successfully


In [49]:
# Tạo schema mapping cho tất cả 6 bảng
print("=== SCHEMA DEFINITION ===")

schema_mappings = {}
tables_data = {
    "dim_order_pages": dim_order_pages_clean,
    "dim_order_warehouses": dim_order_warehouses_clean,
    "dim_order_shipping": dim_order_shipping_clean,
    "dim_order_payments": dim_order_payments_clean,
    "fact_orders": fact_orders_clean,
    "fact_order_items": fact_order_items_clean
}

for table_name, df in tables_data.items():
    schema_mappings[table_name] = create_schema_mapping(df, table_name)

print(f"\n✅ Schema mappings created for {len(schema_mappings)} tables")


=== SCHEMA DEFINITION ===
Schema mapping for dim_order_pages: 4 columns
Schema mapping for dim_order_warehouses: 7 columns
Schema mapping for dim_order_shipping: 7 columns
Schema mapping for dim_order_payments: 8 columns
Schema mapping for fact_orders: 24 columns
Schema mapping for fact_order_items: 11 columns

✅ Schema mappings created for 6 tables


In [50]:
# Load vào Silver database theo thứ tự
print(f"\n=== LOADING TO SILVER DATABASE ===")

# Load dim tables first
dim_tables = ["dim_order_pages", "dim_order_warehouses", "dim_order_shipping", "dim_order_payments"]
for table_name in dim_tables:
    if table_name in tables_data:
        success = load_to_silver(tables_data[table_name], table_name, schema_mappings[table_name])
        if not success:
            print(f"❌ Failed to load {table_name}")

# Load fact tables
fact_tables = ["fact_orders", "fact_order_items"]
for table_name in fact_tables:
    if table_name in tables_data:
        success = load_to_silver(tables_data[table_name], table_name, schema_mappings[table_name])
        if not success:
            print(f"❌ Failed to load {table_name}")

print(f"\n✅ All tables loaded to Silver database successfully!")



=== LOADING TO SILVER DATABASE ===
✅ Đã load 74 records vào Silver: dim_order_pages
✅ Verification: 74 records trong database
✅ Đã load 24 records vào Silver: dim_order_warehouses
✅ Verification: 24 records trong database
✅ Đã load 40,236 records vào Silver: dim_order_shipping
✅ Verification: 40,236 records trong database
✅ Đã load 40,236 records vào Silver: dim_order_payments
✅ Verification: 40,236 records trong database
✅ Đã load 40,236 records vào Silver: fact_orders
✅ Verification: 40,236 records trong database
✅ Đã load 46,611 records vào Silver: fact_order_items
✅ Verification: 46,611 records trong database

✅ All tables loaded to Silver database successfully!


In [51]:
# Final verification
print("=== FINAL VERIFICATION ===")

def final_verification():
    print("--- TABLE COUNTS IN SILVER DATABASE ---")
    
    for table_name in tables_data.keys():
        try:
            count_query = f"SELECT COUNT(*) as count FROM {table_name}"
            result = pd.read_sql(count_query, silver_engine)
            count = result['count'].iloc[0]
            print(f"{table_name}: {count:,} records")
        except Exception as e:
            print(f"❌ Error checking {table_name}: {str(e)}")
    
    print("\n--- SAMPLE DATA VERIFICATION ---")
    
    # Sample fact_orders
    try:
        sample_orders = pd.read_sql("SELECT order_id, status, total_price, customer_id FROM fact_orders LIMIT 3", silver_engine)
        print("Sample fact_orders:")
        print(sample_orders)
    except Exception as e:
        print(f"❌ Error getting sample fact_orders: {str(e)}")
    
    # Sample fact_order_items
    try:
        sample_items = pd.read_sql("SELECT order_item_id, order_id, item_quantity, item_price FROM fact_order_items LIMIT 3", silver_engine)
        print("\nSample fact_order_items:")
        print(sample_items)
    except Exception as e:
        print(f"❌ Error getting sample fact_order_items: {str(e)}")

final_verification()


=== FINAL VERIFICATION ===
--- TABLE COUNTS IN SILVER DATABASE ---
dim_order_pages: 74 records
dim_order_warehouses: 24 records
dim_order_shipping: 40,236 records
dim_order_payments: 40,236 records
fact_orders: 40,236 records
fact_order_items: 46,611 records

--- SAMPLE DATA VERIFICATION ---
Sample fact_orders:
  order_id  status  total_price                           customer_id
0    40616       0          0.0  e906c5c7-ea19-42e5-a971-57cf09c94417
1    40615       0          0.0  6c5b40d2-ae42-4e13-8019-f3f9127d75a0
2    40614       0          0.0  765c531b-743f-4f30-b6b0-0d6f3579630f

Sample fact_order_items:
  order_item_id order_id  item_quantity  item_price
0  ITEM_40458_0    40458              1    629000.0
1  ITEM_40455_0    40455              1    649000.0
2  ITEM_40454_0    40454              1    649000.0


## Bước 8: Data Dictionary Generation

### Mục tiêu
- Tạo Data Dictionary cho tất cả 6 bảng orders đã load vào Silver
- Append vào file Excel Dictionary.xlsx
- Cung cấp thông tin chi tiết về columns, data types, business meanings

### Quy trình
1. **Generate Dictionary** - Tạo dictionary cho từng bảng
2. **Business Meanings** - Định nghĩa ý nghĩa business cho từng column
3. **Append to Excel** - Thêm vào file Dictionary.xlsx
4. **Summary Report** - Báo cáo tổng kết transformation

### Lưu ý quan trọng
- Append vào file Excel thay vì overwrite
- Bao gồm table_name để phân biệt các bảng
- Cung cấp business meanings rõ ràng
- Đảm bảo format nhất quán với các notebooks khác


In [52]:
# Generate Data Dictionary cho tất cả 6 bảng
print("=== GENERATING DATA DICTIONARY ===")

def get_business_meaning(table_name, column_name):
    """Get business meaning for each column"""
    business_meanings = {
        # fact_orders
        "fact_orders": {
            "order_id": "Unique order identifier (primary key)",
            "system_id": "System order ID",
            "shop_id": "Shop identifier (foreign key to dim_shops)",
            "order_link": "Order link URL",
            "link_confirm_order": "Order confirmation link",
            "order_currency": "Order currency (VND)",
            "total_price": "Total order value before discount",
            "total_price_after_sub_discount": "Final order value after discount",
            "total_discount": "Total discount amount",
            "total_quantity": "Total quantity of items",
            "items_length": "Number of different products",
            "tax": "Tax amount",
            "shipping_fee": "Shipping fee",
            "money_to_collect": "Amount to collect from customer",
            "is_free_shipping": "Free shipping flag",
            "customer_id": "Customer identifier (foreign key to dim_customers)",
            "page_id": "Page identifier (foreign key to dim_order_pages)",
            "warehouse_id": "Warehouse identifier (foreign key to dim_order_warehouses)",
            "shipping_id": "Shipping identifier (foreign key to dim_order_shipping)",
            "payment_id": "Payment identifier (foreign key to dim_order_payments)",
            "status": "Order status code",
            "status_name": "Order status name",
            "inserted_at": "Order creation timestamp",
            "updated_at": "Order last update timestamp"
        },
        
        # dim_order_pages
        "dim_order_pages": {
            "page_id": "Page identifier (primary key)",
            "page_name": "Page name",
            "page_username": "Page username",
            "page_created_at": "Page creation timestamp"
        },
        
        # dim_order_warehouses
        "dim_order_warehouses": {
            "warehouse_id": "Warehouse identifier (primary key)",
            "warehouse_name": "Warehouse name",
            "warehouse_address": "Warehouse address",
            "warehouse_full_address": "Full warehouse address",
            "warehouse_phone_number": "Warehouse phone number",
            "warehouse_commune_id": "Warehouse commune ID",
            "warehouse_province_id": "Warehouse province ID"
        },
        
        # dim_order_shipping
        "dim_order_shipping": {
            "shipping_id": "Shipping identifier (primary key)",
            "shipping_full_name": "Receiver full name",
            "shipping_phone": "Receiver phone number",
            "shipping_full_address": "Full shipping address",
            "shipping_commune_id": "Shipping commune ID",
            "shipping_district_id": "Shipping district ID",
            "shipping_province_id": "Shipping province ID"
        },
        
        # dim_order_payments
        "dim_order_payments": {
            "payment_id": "Payment identifier (primary key)",
            "payment_method": "Primary payment method (COD/Cash/Card/MoMo/QR)",
            "payment_amount": "Payment amount",
            "cod": "Cash on delivery amount",
            "cash": "Cash payment amount",
            "charged_by_card": "Card payment amount",
            "charged_by_momo": "MoMo payment amount",
            "charged_by_qrpay": "QR payment amount"
        },
        
        # fact_order_items
        "fact_order_items": {
            "order_item_id": "Order item identifier (primary key)",
            "order_id": "Order identifier (foreign key to fact_orders)",
            "product_id": "Product identifier (foreign key to dim_products)",
            "variation_id": "Product variation identifier",
            "item_quantity": "Item quantity",
            "item_price": "Item unit price",
            "item_discount": "Item discount amount",
            "return_quantity": "Returned quantity",
            "returned_count": "Number of returns",
            "product_name": "Product name",
            "product_detail": "Product variation details"
        }
    }
    
    return business_meanings.get(table_name, {}).get(column_name, "No business meaning defined")

print("✅ Business meanings dictionary defined successfully")


=== GENERATING DATA DICTIONARY ===
✅ Business meanings dictionary defined successfully


In [53]:
# Tạo Data Dictionary cho tất cả 6 bảng
all_dict_data = []

for table_name, df in tables_clean.items():
    print(f"Generating dictionary for {table_name}...")
    
    for col in df.columns:
        col_info = {
            "table_name": table_name,
            "column_name": col,
            "dtype": str(df[col].dtype),
            "sql_type": str(schema_mappings[table_name].get(col, "Not defined")),
            "null_count": df[col].isnull().sum(),
            "null_pct": round(df[col].isnull().mean() * 100, 2),
            "unique_count": df[col].nunique(),
            "sample_values": str(df[col].dropna().unique()[:3].tolist()),
            "business_meaning": get_business_meaning(table_name, col),
            "extraction_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        all_dict_data.append(col_info)

# Tạo DataFrame tổng hợp
data_dictionary = pd.DataFrame(all_dict_data)

# Hiển thị Data Dictionary
print(f"Generated Data Dictionary for {len(data_dictionary)} columns across 6 tables")
print(f"\n=== DATA DICTIONARY SUMMARY ===")
print(data_dictionary.groupby('table_name').size().to_dict())


Generating dictionary for fact_orders...
Generating dictionary for dim_order_pages...
Generating dictionary for dim_order_warehouses...
Generating dictionary for dim_order_shipping...
Generating dictionary for dim_order_payments...
Generating dictionary for fact_order_items...
Generated Data Dictionary for 61 columns across 6 tables

=== DATA DICTIONARY SUMMARY ===
{'dim_order_pages': 4, 'dim_order_payments': 8, 'dim_order_shipping': 7, 'dim_order_warehouses': 7, 'fact_order_items': 11, 'fact_orders': 24}


In [54]:
# Append Data Dictionary vào file Excel
excel_path = "Technical_Document/Dictionary.xlsx"

try:
    from openpyxl import load_workbook
    
    # Kiểm tra file Excel có tồn tại không
    try:
        # Load workbook hiện tại
        wb = load_workbook(excel_path)
        
        # Lấy sheet đầu tiên
        ws = wb.active
        
        # Kiểm tra xem có dữ liệu cũ không
        if ws.max_row > 1:
            print(f"Found existing data in {excel_path}, appending new data...")
        else:
            print(f"File {excel_path} exists but is empty, adding header and data...")
            
    except FileNotFoundError:
        print(f"File {excel_path} not found, creating new file...")
        wb = None
    except Exception as e:
        print(f"Error loading {excel_path}: {str(e)}, creating new file...")
        wb = None
    
    if wb is None:
        # Tạo file mới với header
        data_dictionary.to_excel(excel_path, index=False, sheet_name='Data_Dictionary')
        print(f"✅ Created new file: {excel_path}")
    else:
        # Append vào file hiện tại
        from openpyxl.utils.dataframe import dataframe_to_rows
        
        # Tìm dòng cuối cùng có dữ liệu
        last_row = ws.max_row
        
        # Thêm dữ liệu mới từ dòng tiếp theo
        for r in dataframe_to_rows(data_dictionary, index=False, header=False):
            last_row += 1
            for c_idx, value in enumerate(r, 1):
                ws.cell(row=last_row, column=c_idx, value=value)
        
        # Lưu file
        wb.save(excel_path)
        print(f"✅ Appended {len(data_dictionary)} rows to: {excel_path}")
    
except Exception as e:
    print(f"❌ Error appending to Data Dictionary: {str(e)}")
    # Fallback: tạo file mới
    try:
        data_dictionary.to_excel(excel_path, index=False)
        print(f"✅ Created new file as fallback: {excel_path}")
    except Exception as e2:
        print(f"❌ Error creating fallback file: {str(e2)}")


Found existing data in Technical_Document/Dictionary.xlsx, appending new data...
✅ Appended 61 rows to: Technical_Document/Dictionary.xlsx
